# Zbozinek TD, Perez OD, Wise T, Fanselow M, & Mobbs D

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from theano import scan
import theano.tensor as T
import pymc3 as pm
import theano
import seaborn as sns
import os, sys, subprocess
import pickle

# Load Data

In [ ]:
data = pd.read_csv(os.path.join('../data/', "75_25_Modeling_Data_OS1.csv"))

In [ ]:
data['DV'] = ((data['DV'].values - 1) / 2) - 1

observed_R = data.pivot(columns = 'ID', index = 'trialseq', values = 'DV').values[:, np.newaxis, :] #values.T transposes the data, so you can make trials the first dimension or participants first

# Occasion Setting Model

In [ ]:
def learning_function(stimuli_shown, Λ, λ, training_or_test, prev_V, prev_Vbar, prev_P, prev_N, prev_Phat, prev_Nhat, stimulus_type, OS1_type, α, 
                      iotaP,   
                      iotaPabs,
                      iotaN,
                      iotaNabs,
                     ):
    
    Λbar = T.zeros_like(Λ)
    Λbar = T.inc_subtensor(Λbar[0,:], (prev_V[1,:] > 0) * (1 - Λ[1, :])) #R
    Λbar = T.inc_subtensor(Λbar[1,:], (prev_V[1,:] > 0) * (1 - Λ[1, :])) #s
    Λbar = T.inc_subtensor(Λbar[2,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #T
    Λbar = T.inc_subtensor(Λbar[3,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #u
    Λbar = T.inc_subtensor(Λbar[4,:], (prev_V[1,:] > 0) * (1 - Λ[1, :])) #Rabs
    Λbar = T.inc_subtensor(Λbar[5,:], (prev_V[3,:] > 0) * (1 - Λ[3, :])) #Tabs

  
    #λbar
    λbar = T.zeros_like(Λbar)
    λbar = T.inc_subtensor(λbar[0,:], prev_V[1,:]) #R
    λbar = T.inc_subtensor(λbar[1,:], prev_V[1,:]) #s
    λbar = T.inc_subtensor(λbar[2,:], prev_V[3,:]) #T
    λbar = T.inc_subtensor(λbar[3,:], prev_V[3,:]) #u
    λbar = T.inc_subtensor(λbar[4,:], prev_V[1,:]) #Rabs
    λbar = T.inc_subtensor(λbar[5,:], prev_V[3,:]) #Tabs


    pe_V = λ - prev_V
    pe_Vbar = λbar - prev_Vbar
    pe_P = λ - prev_P
    pe_N = λbar - prev_N
    
    pe_Phat = λ - prev_Phat
    pe_Nhat = λbar - prev_Nhat
    
    #iota
    
    iotaP     = iotaP       *(stimuli_shown[[0,2],:].sum(axis=0) > 0)
    iotaPabs  = iotaPabs    *(stimuli_shown[[4,5],:].sum(axis=0) > 0)
    iotaN     = iotaN       *(stimuli_shown[[0,2],:].sum(axis=0) > 0)
    iotaNabs  = iotaNabs    *(stimuli_shown[[4,5],:].sum(axis=0) > 0)
       
    #γ
    prev_γ1 = prev_V * prev_Vbar


    CS_prev_γ1 = (prev_γ1 * stimuli_shown).sum(axis=0)
    ΔV = Λ * (α) * pe_V
    ΔVbar = Λbar * (α) * pe_Vbar
    ΔP = Λ * (α) * CS_prev_γ1 * pe_P
    ΔN = Λbar * (α) * CS_prev_γ1 * pe_N
    
    ΔPhat = Λ * (α) * CS_prev_γ1 * pe_Phat
    ΔNhat = Λbar * (α) * CS_prev_γ1 * pe_Nhat


    # Only update stimuli that were shown
    ΔV = ΔV * stimuli_shown
    ΔVbar = ΔVbar * stimuli_shown
    ΔP = ΔP * stimuli_shown
    ΔN = ΔN * stimuli_shown
    
    ΔPhat = ΔPhat * stimuli_shown
    ΔNhat = ΔNhat * stimuli_shown
    
    # Update V, Vbar, P, N, P2, N2
    V = T.zeros_like(prev_V)
    Vbar = T.zeros_like(prev_Vbar)
    P = T.zeros_like(prev_P)
    N = T.zeros_like(prev_N)
    
    Phat = T.zeros_like(prev_Phat)
    Nhat = T.zeros_like(prev_Nhat)
    
    # Only update V and Vbar for CSs. Only update P and N for 1st-order OSs.
    V = T.inc_subtensor(V[T.eq(stimulus_type, 1)], prev_V[T.eq(stimulus_type, 1)] + ΔV[T.eq(stimulus_type, 1)] * training_or_test)
    Vbar = T.inc_subtensor(Vbar[T.eq(stimulus_type, 1)], prev_Vbar[T.eq(stimulus_type, 1)] + ΔVbar[T.eq(stimulus_type, 1)] * training_or_test)
    P = T.inc_subtensor(P[T.eq(OS1_type, 1)], prev_P[T.eq(OS1_type, 1)] + ΔP[T.eq(OS1_type, 1)] * (iotaP + iotaNabs) * training_or_test)
    N = T.inc_subtensor(N[T.eq(OS1_type, 1)], prev_N[T.eq(OS1_type, 1)] + ΔN[T.eq(OS1_type, 1)] * (iotaN + iotaPabs) * training_or_test)
    
    Phat = T.inc_subtensor(Phat[T.eq(stimulus_type, 1)], prev_Phat[T.eq(stimulus_type, 1)] + ΔPhat[T.eq(stimulus_type, 1)] * training_or_test)
    Nhat = T.inc_subtensor(Nhat[T.eq(stimulus_type, 1)], prev_Nhat[T.eq(stimulus_type, 1)] + ΔNhat[T.eq(stimulus_type, 1)] * training_or_test)
    
    return V, Vbar, P, N, Phat, Nhat

# Generate Simulated Data with Model

In [ ]:
n_stim = 6
n_subjects = len(data['ID'].unique())

#Initial values
R = np.zeros((n_stim, n_subjects))
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects))
v_inhibitory = np.zeros((n_stim, n_subjects))
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))

Phat = np.zeros((n_stim, n_subjects))
Nhat = np.zeros((n_stim, n_subjects))


#Randomized parameter values 
gen_dist = pm.Beta.dist(2, 2, shape = n_subjects)
α_subject_sim = gen_dist.random()

iotaP_subject_sim     = gen_dist.random()
iotaPabs_subject_sim  = gen_dist.random()
iotaN_subject_sim     = gen_dist.random()
iotaNabs_subject_sim  = gen_dist.random()


#Test vs Training Trial
training_or_test = data.pivot(index='trialseq', values='Test', columns='ID').values[:, np.newaxis, :].astype(float)

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['R', 's', 'T', 'u', 'R_abs', 'T_abs']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[0, 2, 4, 5]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[0, 2, 4, 5]] = 1 #make 1st OSs = 1

#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)
training_or_test = T.as_tensor_variable(training_or_test)

stimuli_shown_sim = stimuli_shown.copy()
big_lambda_sim = big_lambda.copy()
small_lambda_sim = small_lambda.copy()
training_or_test_sim = training_or_test.copy()

# Run Fake Data Simulation

In [ ]:
#Run the loop
output, updates = scan(fn=learning_function,
                    sequences=[{'input': stimuli_shown_sim[:-1, ...]},
                             {'input': big_lambda_sim},
                             {'input': small_lambda_sim},
                              {'input': training_or_test}],
                    outputs_info=[v_excitatory, v_inhibitory, P, N, Phat, Nhat],
                    non_sequences = [stimulus_type, OS1_type, α_subject_sim, 
                                     iotaP_subject_sim,   
                                     iotaPabs_subject_sim,
                                     iotaN_subject_sim,    
                                     iotaNabs_subject_sim])

#Get model output
V_out, Vbar_out, P_out, N_out, Phat_out, Nhat_out = [i.eval() for i in output]

estimated_overall_R = ((V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) - (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) + \
    ((P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Phat_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) - \
    ((N_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Nhat_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1))

overall_R_sim = estimated_overall_R.eval()

# Check parameter recovery

In [ ]:
n_subjects = len(data['ID'].unique())
n_stim = 6

#Initial values
R = np.zeros((n_stim, n_subjects))

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['R', 's', 'T', 'u', 'R_abs', 'T_abs']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[0, 2, 4, 5]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[0, 2, 4, 5]] = 1 #make 1st OSs = 1

#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1
    α_mean        = pm.Normal    ('α_mean', 0.5, 10)
    α_sd          = pm.HalfCauchy('α_sd', 10)
    
    iotaP_mean    = pm.Normal    ('iotaP_mean', 0.5, 10)
    iotaP_sd      = pm.HalfCauchy('iotaP_sd', 10)
    iotaPabs_mean = pm.Normal    ('iotaPabs_mean', 0.5, 10)
    iotaPabs_sd   = pm.HalfCauchy('iotaPabs_sd', 10)    
    iotaN_mean    = pm.Normal    ('iotaN_mean', 0.5, 10)
    iotaN_sd      = pm.HalfCauchy('iotaN_sd', 10)
    iotaNabs_mean = pm.Normal    ('iotaNabs_mean', 0.5, 10)
    iotaNabs_sd   = pm.HalfCauchy('iotaNabs_sd', 10) 
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    α_subject = BoundedNormal('α', mu=α_mean, sd=α_sd, shape=(n_subjects,))
    
    iotaP_subject       = BoundedNormal('iotaP'   ,    mu=iotaP_mean,    sd=iotaP_sd,    shape=(n_subjects,))
    iotaPabs_subject    = BoundedNormal('iotaPabs',    mu=iotaPabs_mean, sd=iotaPabs_sd, shape=(n_subjects,))
    iotaN_subject       = BoundedNormal('iotaN'   ,    mu=iotaN_mean,    sd=iotaN_sd,    shape=(n_subjects,))
    iotaNabs_subject    = BoundedNormal('iotaNabs' ,    mu=iotaNabs_mean, sd=iotaNabs_sd, shape=(n_subjects,))
    
    # Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[{'input': stimuli_shown[:-1, ...]},
                             {'input': big_lambda},
                             {'input': small_lambda},
                                {'input': training_or_test}],
                      outputs_info=[v_excitatory, v_inhibitory, P, N, Phat, Nhat],
                      non_sequences=[stimulus_type, OS1_type, α_subject, 
                                     iotaP_subject, 
                                     iotaPabs_subject, 
                                     iotaN_subject, 
                                     iotaNabs_subject
                                    ])
    
    # Get model output
    V, Vbar, P, N, Phat, Nhat = output

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Phat * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (Nhat * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1))
   
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(overall_R_sim.squeeze()))

# Fit the Model

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=25000, callbacks=[CheckParametersConvergence()])
trace = approx.sample(1000)

In [ ]:
parameter_output = pm.summary(trace, kind='stats', var_names=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i
                                                         or 'iota' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

In [ ]:
recovered_data_var = {'Simulated_α'       :    α_subject_sim,           'Recovered_α'          : trace['α']          .mean(axis=0),
                      'Simulated_iotaP'   :    iotaP_subject_sim,       'Recovered_iotaP'      : trace['iotaP']      .mean(axis=0),
                      'Simulated_iotaPabs':    iotaPabs_subject_sim,    'Recovered_iotaP_bs'   : trace['iotaPabs']   .mean(axis=0),
                      'Simulated_iotaN'   :    iotaN_subject_sim,       'Recovered_iotaN'      : trace['iotaN']      .mean(axis=0),
                      'Simulated_iotaNabs':    iotaNabs_subject_sim,    'Recovered_iotaNabs'   : trace['iotaNabs']   .mean(axis=0),
}


recovered_data_var = pd.DataFrame(recovered_data_var)
recovered_data_var.to_csv(os.path.join('../output/',r'75_25_OS1_Iota_Simulated_vs_Recovered.csv'))

In [ ]:
f, ax = plt.subplots(3, 2, sharex = True, sharey = True, figsize=(12, 7.5))
f.suptitle('Simulated vs Recovered α Parameters', y=1.02, fontsize = 16)
f.text(.5, -.02, 'Simulated', va='center', ha='center', fontsize = 16)
f.text(-.02, .5, 'Recovered', va='center', ha='center', fontsize = 16, rotation=90)

sns.regplot(α_subject_sim,        trace['α']       .mean(axis=0), label='α_subject',        ax=ax[0,0], color = 'black')
sns.regplot(iotaP_subject_sim,    trace['iotaP']   .mean(axis=0), label='iotaP_subject',    ax=ax[0,1], color = 'black')
sns.regplot(iotaPabs_subject_sim, trace['iotaPabs'].mean(axis=0), label='iotaPabs_subject', ax=ax[1,0], color = 'black')
sns.regplot(iotaN_subject_sim,    trace['iotaN']   .mean(axis=0), label='iotaN_subject',    ax=ax[1,1], color = 'black')
sns.regplot(iotaNabs_subject_sim, trace['iotaNabs'].mean(axis=0), label='iotaNabs_subject', ax=ax[2,0], color = 'black')


for i in range(3):
    for j in range(3):
        ax[0,0].set_title('α')
        ax[0,1].set_title('iotaP')
        ax[1,0].set_title('iotaPabs')
        ax[1,1].set_title('iotaN')
        ax[2,0].set_title('iotaNabs')


plt.setp(ax, xticks=[0, .2, .4, .6, .8, 1], yticks=[0, .2, .4, .6, .8, 1])        
plt.tight_layout()

plt.savefig(os.path.join('../output/',r'75_25_OS1_Iota_Simulated_vs_Recovered.svg'), bbox_inches='tight')

# Fit the Model to Real Data

In [ ]:
n_subjects = len(data['ID'].unique())
n_stim = 6

# Initial values
R = np.zeros((n_stim, n_subjects))  # Value estimate
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects)) 
v_inhibitory = np.zeros((n_stim, n_subjects)) 
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))

Phat = np.zeros((n_stim, n_subjects))
Nhat = np.zeros((n_stim, n_subjects))

# US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['R', 's', 'T', 'u', 'R_abs', 'T_abs']].values)
    
stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

# Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]])  # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[0, 2, 4, 5]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[0, 2, 4, 5]] = 1 #make 1st OSs = 1

# Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda)
small_lambda = T.as_tensor_variable(small_lambda)
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1
    α_mean        = pm.Normal    ('α_mean', 0.5, 10)
    α_sd          = pm.HalfCauchy('α_sd', 10)
    
    iotaP_mean    = pm.Normal    ('iotaP_mean', 0.5, 10)
    iotaP_sd      = pm.HalfCauchy('iotaP_sd', 10)
    iotaPabs_mean = pm.Normal    ('iotaPabs_mean', 0.5, 10)
    iotaPabs_sd   = pm.HalfCauchy('iotaPabs_sd', 10)    
    iotaN_mean    = pm.Normal    ('iotaN_mean', 0.5, 10)
    iotaN_sd      = pm.HalfCauchy('iotaN_sd', 10)
    iotaNabs_mean = pm.Normal    ('iotaNabs_mean', 0.5, 10)
    iotaNabs_sd   = pm.HalfCauchy('iotaNabs_sd', 10) 
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    α_subject = BoundedNormal('α', mu=α_mean, sd=α_sd, shape=(n_subjects,))
    
    iotaP_subject       = BoundedNormal('iotaP'   ,    mu=iotaP_mean,    sd=iotaP_sd,    shape=(n_subjects,))
    iotaPabs_subject    = BoundedNormal('iotaPabs',    mu=iotaPabs_mean, sd=iotaPabs_sd, shape=(n_subjects,))
    iotaN_subject       = BoundedNormal('iotaN'   ,    mu=iotaN_mean,    sd=iotaN_sd,    shape=(n_subjects,))
    iotaNabs_subject    = BoundedNormal('iotaNabs' ,   mu=iotaNabs_mean, sd=iotaNabs_sd, shape=(n_subjects,))
    
    # Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[{'input': stimuli_shown[:-1, ...]},
                             {'input': big_lambda},
                             {'input': small_lambda},
                                {'input': training_or_test}],
                      outputs_info=[v_excitatory, v_inhibitory, P, N, Phat, Nhat],
                      non_sequences=[stimulus_type, OS1_type, α_subject, 
                                     iotaP_subject, 
                                     iotaPabs_subject, 
                                     iotaN_subject, 
                                     iotaNabs_subject
                                    ])
    
    # Get model output
    V, Vbar, P, N, Phat, Nhat = output

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Phat * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (Nhat * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1))
    
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    V = pm.Deterministic('estimated_V', V)
    Vbar = pm.Deterministic('estimated_Vbar', Vbar)
    P = pm.Deterministic('estimated_P', P)
    N = pm.Deterministic('estimated_N', N)
    γ1 = pm.Deterministic('estimated_γ1', V*Vbar)
    
    Phat = pm.Deterministic('estimated_Phat', Phat)
    Nhat = pm.Deterministic('estimated_Nhat', Nhat)
          
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(observed_R.squeeze()))

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=25000, callbacks=[CheckParametersConvergence()])

In [ ]:
trace = approx.sample(1000)

In [ ]:
#This saves the results of the modeling
with open('75_25_OS1_Iota.pkl', 'wb') as f:
    pickle.dump(trace, f)

In [ ]:
#This loads the saved results of the modeling
trace = pickle.load(open('75_25_OS1_Iota.pkl', 'rb'))

In [ ]:
parameter_output = pm.summary(trace, kind='stats', var_names=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i
                                                         or 'iota' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

# Model Output

In [ ]:
overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)
sub_ids = data['ID'].unique()
subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]

In [ ]:
waic_output = pm.waic(trace)

In [ ]:
waic_output

In [ ]:
parameter_output.to_csv(os.path.join('../output/',r'75_25_OS1_Iota_Alpha.csv'))
waic_output.to_csv(os.path.join('../output/',r'75_25_OS1_Iota_WAIC.csv'))

In [ ]:
f, ax = plt.subplots(10, 2, figsize=(36, 48), dpi = 100)

overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)

sub_ids = data['ID'].unique()

subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]
    

for n, sub in enumerate(subs):
    ax[n % 10, int(n / 10)].fill_between(range(overall_R_mean.shape[0]), overall_R_mean[:, sub] - overall_R_sd[:, sub], overall_R_mean[:, sub] + overall_R_sd[:, sub], alpha=0.3)
    ax[n % 10, int(n / 10)].plot(overall_R_mean[:, sub])
    ax[n % 10, int(n / 10)].plot(observed_R.squeeze()[:, sub], color='orange', linestyle='-')#participant's real data
    if n == 0:
        ax[n % 10, int(n / 10)].set_ylabel('Mean (+/-SD) overall R')
    ax[n % 10, int(n / 10)].set_ylabel('Responding (R)')
    ax[n % 10, int(n / 10)].set_xlabel('Trials')
    ax[n % 10, int(n / 10)].set_title('Sub {0}'.format(sub_ids[n]))   

plt.tight_layout()
plt.savefig(os.path.join('../output/',r'75_25_OS1_Iota_Individual_Real_and_Estimated_Responding.svg'), bbox_inches='tight')

In [ ]:
%load_ext watermark
%watermark -v -p pytest,jupyterlab,numpy,pandas,theano,pymc3